# DOLFINx in Parallel with MPI

To run these examples, first you will need to install the mpi4py library by running this code (this will usually take a while to install the first time):

In [ ]:
! pip install mpi4py

## First parallel program

In [ ]:
%%writefile hello-world.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
print(f"Hello world from rank {comm.rank} of {comm.size}")

Next we see how we can use the mpirun program to execute the above python code using 4 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [ ]:
! mpirun --allow-run-as-root -np 4 python3 hello_world.py

# Point-to-Point Communication

In [ ]:
%%writefile p2p.py

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
assert comm.size == 2
rank = comm.rank

if rank == 0:
    data = numpy.arange(10, dtype=int)
    comm.Send(data, dest=1)
    print(f"Process 0 sent {data} to process 1")
elif rank == 1:
    data = numpy.zeros(10, dtype=int)
    comm.Recv(data, source=0)
    print(f"Process 1 received {data} from process 0")


In [ ]:
!mpirun -n 2 python3 p2p.py

MPI can do a lot more than this (https://mpi4py.readthedocs.io/en/stable/tutorial.html). The key points are:
 - N identical versions of your program run, one on each process (rank). Each process takes different paths through the program depending on its rank.
 - Processes (and hence their memory) are totally independent.
 - Communication between processes is must be manually performed by the programmer (explicit).